In [1]:
import time
start = time.time()

import os
import re
import pickle
import pandas as pd
from datetime import timedelta
from unicodedata import normalize, combining

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='luizhemelo', timeout=10)

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

PATH = "../../dados_enem"  # path to datasets
cols = ['NO_MUNICIPIO_PROVA', 'CO_MUNICIPIO_PROVA', 'TP_COR_RACA']  # cities col reference

## Funções auxiliares

In [2]:
# city location cache
path = "../data/cities.pickle"
if os.path.exists(path):
    file = open(path, 'rb')
    cities = pickle.load(file)
    file.close()
else:
    cities = {
        'poxereu':[-15.8290891, -54.3925375, "mato grosso"],
        'brasilia':[-10.3333333, -53.2, "distrito federal"]
    }

def format_name(name: str):
    """
    Format name to remove punctuation and spaces
    """
    name = re.sub("([^a-zà-üA-ZÀ-Ü0-9])", " ", name)
    nfkd_str = normalize("NFKD", name)
    name = "".join(
        [c for _, c in enumerate(nfkd_str) if not combining(c)]
    ).lower()
    while "  " in name:
        name = name.replace("   ", " ")
        name = name.replace("  ", " ")
    return name

def preprocess_dataset(df: pd.DataFrame, year: int):
    occ = df['NO_MUNICIPIO_PROVA'].value_counts().values
    ctt = df['NO_MUNICIPIO_PROVA'].value_counts().index.tolist()

    cor_brancos = []
    cor_pretos = []
    ibge_id = []
    
    for city in ctt:
        df_temp = df[df["NO_MUNICIPIO_PROVA"]==city].reset_index(drop=True)
        value_counts = df_temp["TP_COR_RACA"].value_counts()
        try:
            cor_brancos.append(value_counts[1])
        except:
            cor_brancos.append(0)
        try:
            cor_pretos.append(value_counts[2])
        except:
            cor_pretos.append(0)
        ibge_id.append(df_temp["CO_MUNICIPIO_PROVA"][0])

    df = pd.DataFrame({
        'Ano':year,
        'Num_Candidatos':occ,
        'Num_Brancos':cor_brancos,
        'Num_Pretos':cor_pretos,
        'id':ibge_id,
        'Cidade':ctt
    })
    latitude = []
    longitude = []
    states = []
    
    for i,row in df.iterrows():
        city = format_name(row['Cidade'])
        df.loc[i,'Cidade'] = city
        
        if city not in list(cities.keys()):
            location = geolocator.geocode(str(city+',Brazil'))
            match = geolocator.reverse(location.raw['lat']+","+location.raw['lon'])
            address = match.raw['address']
            state = format_name(address.get('state',''))
            
            cities[city] = [location.raw['lat'], location.raw['lon'], state]
            
            latitude.append(location.raw['lat'])
            longitude.append(location.raw['lon'])
            states.append(state)
        else:
            latitude.append(cities[city][0])
            longitude.append(cities[city][1])
            states.append(cities[city][2])
    
    df['Estado'] = states
    df['Latitude'] = latitude
    df['Longitude'] = longitude
    
    return df

## Carregamento dos dados e pré-processamento

In [3]:
# 01
enem13 = pd.read_csv(f"{PATH}/microdados_enem_2013/DADOS/MICRODADOS_ENEM_2013.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem13 = preprocess_dataset(enem13, 2013)
enem13

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2013,271845,144046,29765,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2013,200306,89097,32028,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2013,167533,45373,13575,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2013,141551,17239,51917,2927408,salvador,bahia,-12.9822499,-38.4812772
4,2013,136953,45891,23686,3106200,belo horizonte,minas gerais,-19.9227318,-43.9450948
...,...,...,...,...,...,...,...,...,...
1608,2013,153,13,8,1302306,jutai,amazonas,-1.72014,-66.8877467
1609,2013,148,12,3,1300029,alvaraes,amazonas,-3.2155787,-64.8148327
1610,2013,146,10,2,1400159,bonfim,roraima,3.3613818,-59.8427241
1611,2013,121,38,14,3113800,carmesia,minas gerais,-19.0854535,-43.1392003


In [4]:
# 02
enem14 = pd.read_csv(f"{PATH}/microdados_enem_2014/DADOS/MICRODADOS_ENEM_2014.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem14 = preprocess_dataset(enem14, 2014)
enem14

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2014,324056,164488,38118,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2014,242222,101969,41107,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2014,188730,46249,16273,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2014,180008,20609,67373,2927408,salvador,bahia,-12.9822499,-38.4812772
4,2014,164363,53120,29485,3106200,belo horizonte,minas gerais,-19.9227318,-43.9450948
...,...,...,...,...,...,...,...,...,...
1697,2014,174,52,15,5103908,general carneiro,parana,-26.422982,-51.3146691
1698,2014,171,39,22,3113800,carmesia,minas gerais,-19.0854535,-43.1392003
1699,2014,168,13,7,1302108,japura,amazonas,-7.1781505,-71.8538963
1700,2014,146,17,3,1304260,uarini,amazonas,-2.9827061,-65.1578615


In [5]:
# 03
enem15 = pd.read_csv(f"{PATH}/microdados_enem_2015/DADOS/MICRODADOS_ENEM_2015.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem15 = preprocess_dataset(enem15, 2015)
enem15

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2015,318507,156239,39688,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2015,215303,89273,36770,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2015,162206,45724,24337,5300108,brasilia,distrito federal,-10.333333,-53.2
3,2015,161869,36257,14821,2304400,fortaleza,ceara,-3.7304512,-38.5217989
4,2015,157824,17790,60254,2927408,salvador,bahia,-12.9822499,-38.4812772
...,...,...,...,...,...,...,...,...,...
1683,2015,117,14,2,1302108,japura,amazonas,-7.1781505,-71.8538963
1684,2015,113,18,3,1300904,canutama,amazonas,-6.5330019,-64.3834966
1685,2015,89,14,4,1301951,itamarati,amazonas,-6.4385189,-68.2437385
1686,2015,85,12,3,1200435,santa rosa do purus,acre,-9.4354872,-70.4926109


In [6]:
# 04
enem16 = pd.read_csv(f"{PATH}/microdados_enem_2016/DADOS/MICRODADOS_ENEM_2016.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem16 = preprocess_dataset(enem16, 2016)
enem16

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2016,364747,175220,48406,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2016,210604,86160,37623,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2016,174685,37727,16385,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2016,172706,18441,67070,2927408,salvador,bahia,-12.9822499,-38.4812772
4,2016,167826,46495,25783,5300108,brasilia,distrito federal,-10.333333,-53.2
...,...,...,...,...,...,...,...,...,...
1687,2016,153,16,3,1304260,uarini,amazonas,-2.9827061,-65.1578615
1688,2016,141,21,3,1200435,santa rosa do purus,acre,-9.4354872,-70.4926109
1689,2016,140,18,11,1400159,bonfim,roraima,3.3613818,-59.8427241
1690,2016,140,24,5,1301951,itamarati,amazonas,-6.4385189,-68.2437385


In [7]:
# 05
enem17 = pd.read_csv(f"{PATH}/microdados_enem_2017/DADOS/MICRODADOS_ENEM_2017.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem17 = preprocess_dataset(enem17, 2017)
enem17

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2017,302110,145392,40241,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2017,171221,71874,29852,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2017,125744,28392,11966,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2017,125257,36153,18909,5300108,brasilia,distrito federal,-10.333333,-53.2
4,2017,120793,19167,7664,1302603,manaus,amazonas,-3.1316333,-59.9825041
...,...,...,...,...,...,...,...,...,...
1685,2017,137,31,3,1300904,canutama,amazonas,-6.5330019,-64.3834966
1686,2017,126,7,3,1304260,uarini,amazonas,-2.9827061,-65.1578615
1687,2017,111,19,0,1301951,itamarati,amazonas,-6.4385189,-68.2437385
1688,2017,94,11,3,1200435,santa rosa do purus,acre,-9.4354872,-70.4926109


In [8]:
# 06
enem18 = pd.read_csv(f"{PATH}/microdados_enem_2018/DADOS/MICRODADOS_ENEM_2018.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem18 = preprocess_dataset(enem18, 2018)
enem18

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2018,247581,122510,31450,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2018,150866,64688,25809,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2018,111056,25664,10126,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2018,106304,32020,15346,5300108,brasilia,distrito federal,-10.333333,-53.2
4,2018,98780,34064,16507,3106200,belo horizonte,minas gerais,-19.9227318,-43.9450948
...,...,...,...,...,...,...,...,...,...
1685,2018,84,5,0,1302108,japura,amazonas,-7.1781505,-71.8538963
1686,2018,77,6,4,1400159,bonfim,roraima,3.3613818,-59.8427241
1687,2018,65,9,5,1200328,jordao,acre,-9.1905396,-71.9484803
1688,2018,54,3,1,1200435,santa rosa do purus,acre,-9.4354872,-70.4926109


In [9]:
# 07
enem19 = pd.read_csv(f"{PATH}/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem19 = preprocess_dataset(enem19, 2019)
enem19

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2019,216016,109985,26978,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2019,134778,60400,22803,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2019,95849,30595,13815,5300108,brasilia,distrito federal,-10.333333,-53.2
3,2019,95089,23834,8669,2304400,fortaleza,ceara,-3.7304512,-38.5217989
4,2019,87894,12031,32108,2927408,salvador,bahia,-12.9822499,-38.4812772
...,...,...,...,...,...,...,...,...,...
1687,2019,90,6,1,1302801,maraa,amazonas,-1.8535579,-65.5897749
1688,2019,57,2,1,1302108,japura,amazonas,-7.1781505,-71.8538963
1689,2019,52,7,1,1400159,bonfim,roraima,3.3613818,-59.8427241
1690,2019,49,25,2,2605459,fernando de noronha,pernambuco,-3.8537498,-32.41980180663255


In [10]:
# 08
enem20 = pd.read_csv(f"{PATH}/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem20 = preprocess_dataset(enem20, 2020)
enem20

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2020,250475,122047,34510,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2020,156727,66045,29069,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2020,116932,36205,17853,5300108,brasilia,distrito federal,-10.333333,-53.2
3,2020,114037,18090,7234,1302603,manaus,amazonas,-3.1316333,-59.9825041
4,2020,108508,26815,9752,2304400,fortaleza,ceara,-3.7304512,-38.5217989
...,...,...,...,...,...,...,...,...,...
1707,2020,129,1,1,1302108,japura,amazonas,-7.1781505,-71.8538963
1708,2020,122,9,9,1400159,bonfim,roraima,3.3613818,-59.8427241
1709,2020,115,14,16,1200328,jordao,acre,-9.1905396,-71.9484803
1710,2020,77,8,5,1200435,santa rosa do purus,acre,-9.4354872,-70.4926109


In [11]:
# 09
enem21 = pd.read_csv(f"{PATH}/microdados_enem_2021/DADOS/MICRODADOS_ENEM_2021.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem21 = preprocess_dataset(enem21, 2021)
enem21

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2021,141840,78426,17134,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2021,102152,50271,17032,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2021,72457,21025,5480,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2021,67501,26097,9126,5300108,brasilia,distrito federal,-10.333333,-53.2
4,2021,58502,10614,21050,2927408,salvador,bahia,-12.9822499,-38.4812772
...,...,...,...,...,...,...,...,...,...
1707,2021,53,18,4,3505401,barra do turvo,sao paulo,-24.756421,-48.5056942
1708,2021,47,18,9,2605459,fernando de noronha,pernambuco,-3.8537498,-32.41980180663255
1709,2021,44,4,2,1200328,jordao,acre,-9.1905396,-71.9484803
1710,2021,43,3,2,1400159,bonfim,roraima,3.3613818,-59.8427241


In [12]:
# 10
enem22 = pd.read_csv(f"{PATH}/microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem22 = preprocess_dataset(enem22, 2022)
enem22

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2022,147046,81901,17383,3550308,sao paulo,sao paulo,-23.5506507,-46.6333824
1,2022,105397,52041,17382,3304557,rio de janeiro,rio de janeiro,-22.9110137,-43.2093727
2,2022,71714,23057,5501,2304400,fortaleza,ceara,-3.7304512,-38.5217989
3,2022,65401,26138,8500,5300108,brasilia,distrito federal,-10.333333,-53.2
4,2022,59976,15545,8399,1501402,belem,para,-1.45056,-48.4682453
...,...,...,...,...,...,...,...,...,...
1707,2022,57,6,2,1200328,jordao,acre,-9.1905396,-71.9484803
1708,2022,53,5,19,1703701,brejinho de nazare,tocantins,-11.0138265,-48.564626
1709,2022,51,18,6,3505401,barra do turvo,sao paulo,-24.756421,-48.5056942
1710,2022,48,5,2,1302108,japura,amazonas,-7.1781505,-71.8538963


## Finalização da base de dados final

In [13]:
if os.path.exists(path) == False:
    file = open(path, 'wb')
    pickle.dump(cities, file)
    file.close()

final = pd.concat([enem13,enem14,enem15,enem16,enem17,enem18,enem19,enem20,enem21,enem22]).reset_index(drop=True)
final[["Latitude", "Longitude"]] = final[["Latitude", "Longitude"]].apply(pd.to_numeric)

final.to_pickle(f"{PATH}/enem_data.pickle")
final.to_csv(f"{PATH}/enem_data.csv")

final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16903 entries, 0 to 16902
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ano             16903 non-null  int64  
 1   Num_Candidatos  16903 non-null  int64  
 2   Num_Brancos     16903 non-null  int64  
 3   Num_Pretos      16903 non-null  int64  
 4   id              16903 non-null  int64  
 5   Cidade          16903 non-null  object 
 6   Estado          16903 non-null  object 
 7   Latitude        16903 non-null  float64
 8   Longitude       16903 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 1.2+ MB


In [14]:
final

,Ano,Num_Candidatos,Num_Brancos,Num_Pretos,id,Cidade,Estado,Latitude,Longitude
0,2013,271845,144046,29765,3550308,sao paulo,sao paulo,-23.550651,-46.633382
1,2013,200306,89097,32028,3304557,rio de janeiro,rio de janeiro,-22.911014,-43.209373
2,2013,167533,45373,13575,2304400,fortaleza,ceara,-3.730451,-38.521799
3,2013,141551,17239,51917,2927408,salvador,bahia,-12.982250,-38.481277
4,2013,136953,45891,23686,3106200,belo horizonte,minas gerais,-19.922732,-43.945095
...,...,...,...,...,...,...,...,...,...
16898,2022,57,6,2,1200328,jordao,acre,-9.190540,-71.948480
16899,2022,53,5,19,1703701,brejinho de nazare,tocantins,-11.013827,-48.564626
16900,2022,51,18,6,3505401,barra do turvo,sao paulo,-24.756421,-48.505694
16901,2022,48,5,2,1302108,japura,amazonas,-7.178151,-71.853896


In [15]:
print("Elapsed time (h:mm:ss):",str(timedelta(seconds=(time.time() - start))))

Elapsed time (h:mm:ss): 2:00:03.856334
